In [23]:
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
df_raw = pd.read_csv("billing_08-246800072_2023_03-2023_06.csv")
df_raw = df_raw.drop(["Workspace ID", "License Settings", "SKU"], axis=1)
df_compute = df_raw[df_raw["Type"] == 'Compute'].reset_index(drop=True)
df_compute = df_compute.drop(["Type","Submit Date", "Start Date", "Stop Date", "Unit Charge/Hour", "Walltime"], axis=1)
df_compute['Coretype'] = df_compute['Description'].str.extract(r':\s(.*)')
df_compute['Charge'] = df_compute['Charge'].str.replace("$", "")
df_compute['Charge'] = df_compute['Charge'].astype(float)
df_compute = df_compute.drop(["Description"], axis=1)
column_order = ["Month", "Project ID", "Project Name", "User", "Job Type", "ID", "Name",
                "Software", "Rescale On-Demand License",
                "Coretype", "Billing Priority", "Cores", "Unit Hours", "Charge"]
df_compute = df_compute[column_order]
df_compute.columns = ["Month", "Project ID", "Project Name", "User", "Job Type", "Job ID", "Job Name",
                      "Software", "Rescale On-Demand License",
                      "Coretype", "Billing Priority", "Cores", "Unit Hours", "Charge"]
#df_compute["Charge"] = df_compute["Charge"].str.replace("$", "")
print(df_compute.head(10))

     Month Project ID    Project Name                 User Job Type  Job ID  \
0  2023-04      UgQWc  Welding_ABAQUS  dohyunko@hanwha.com    Basic  jGhHhb   
1  2023-04      UgQWc  Welding_ABAQUS  dohyunko@hanwha.com    Basic  tSsLeb   
2  2023-04      UgQWc  Welding_ABAQUS  dohyunko@hanwha.com    Basic   DMTgU   
3  2023-04      UgQWc  Welding_ABAQUS  dohyunko@hanwha.com    Basic   zBTgU   
4  2023-04      UgQWc  Welding_ABAQUS  dohyunko@hanwha.com    Basic   OMTgU   
5  2023-04      UgQWc  Welding_ABAQUS  dohyunko@hanwha.com    Basic   endmU   
6  2023-04      UgQWc  Welding_ABAQUS  dohyunko@hanwha.com    Basic   FrTNm   
7  2023-04      UgQWc  Welding_ABAQUS  dohyunko@hanwha.com    Basic   owujb   
8  2023-04      UgQWc  Welding_ABAQUS  dohyunko@hanwha.com    Basic   swexp   
9  2023-04      UgQWc  Welding_ABAQUS  dohyunko@hanwha.com    Basic   VgpBp   

                                            Job Name Software  \
0                                           작업 제목 없음   Abaqus   


/tmp/ipykernel_20798/427950085.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_compute['Charge'] = df_compute['Charge'].str.replace("$", "")


In [25]:
import os
import requests
import json

home_path = os.path.expanduser("~")
apiconfig = home_path + '/' + '.config' + '/' + 'rescale' + '/' + 'apiconfig'
if (os.path.isfile(apiconfig)):
    f = open(apiconfig, "r")
    lines = f.readlines()
    f.close()
    apibaseurl = lines[1].split("=")[1].rstrip('\n').lstrip().replace("'","")
    apikey = lines[2].split("=")[1].rstrip('\n').lstrip().replace("'","")
    token = 'Token ' + apikey
    print('Reading the apiconfig file is done')


current_page = 1
last_page = False


precheck_count = requests.get(
    apibaseurl + '/api/v2/jobs/',
    headers = {'Authorization': token}
)

precheck_count_dict = json.loads(precheck_count.text)
count = precheck_count_dict['count']
print(count)

jobid = [0 for i in range(count)]

while (not(last_page)):
    check_jobid = requests.get(
        apibaseurl + '/api/v2/jobs/',
        headers = {'Authorization': token},
        params = {'page': current_page}
    )
    check_jobid_dict = json.loads(check_jobid.text)

    if current_page == 1:
        for i in range(0, 10):
            jobid[i] = check_jobid_dict['results'][i]['id']
    elif count > current_page * 10:
        indexup = (current_page - 1) * 10
        for i in range(0, 10):
            jobid[i + indexup] = check_jobid_dict['results'][i]['id']
    else:
        indexup = (current_page - 1) * 10
        uplimit = count % indexup
        for i in range(0, uplimit):
            jobid[i + indexup] = check_jobid_dict['results'][i]['id']
    
    if (check_jobid_dict['next'] == None):
        last_page = True
    else:
        current_page += 1

print(jobid)

json_data = json.dumps(check_jobid_dict, indent=4)
with open('overview.json', "w") as file:
    file.write(json_data)

Reading the apiconfig file is done
128
['qCJPd', 'HeYHW', 'fYhNW', 'HgCVW', 'pLhNW', 'rExkeb', 'ccEyT', 'ORDyT', 'Qhdceb', 'quZmW', 'miFdW', 'OXEdW', 'MZoMgb', 'xxvZV', 'knvZV', 'qAYnm', 'bYeBa', 'OkWbd', 'PgFYgb', 'NzACT', 'ABjkeb', 'ZOqyT', 'udguT', 'CFZfeb', 'vdguT', 'HFZfeb', 'xPZfeb', 'xBWpT', 'LSEYgb', 'uqMmW', 'svckd', 'Ysckd', 'OJBOo', 'zgJbd', 'fFvXeb', 'XHRKeb', 'vRhcU', 'xUNUW', 'SiDQW', 'vknyhb', 'ianyhb', 'Qzcuhb', 'rFRKeb', 'Vhnyhb', 'svtMW', 'fYXXT', 'YNrfd', 'dQtrm', 'VpkmW', 'mrahW', 'wjJjeb', 'hHPxT', 'QUFYV', 'kHzLgb', 'rVSoeb', 'mTgbd', 'VTtrm', 'GrAEa', 'JJtrm', 'ugAEa', 'zaMod', 'xcvWo', 'iABjd', 'kSuWo', 'WpBjd', 'vcvWo', 'BvaOo', 'MFaOo', 'zvaOo', 'kTgbd', 'UTtrm', 'FrAEa', 'HJtrm', 'tqaOo', 'eOgbd', 'OOtrm', 'zmAEa', 'cwPdm', 'odMod', 'qvEap', 'bTLod', 'TSLod', 'AsBjd', 'xsBjd', 'JAnbb', 'zQBBp', 'OJSYgb', 'sbxkeb', 'LapFa', 'nnPOo', 'yHVbd', 'JSVbd', 'wwyQgb', 'EREdW', 'jxGrm', 'EMVjeb', 'NmBhW', 'wSSib', 'Hztrm', 'esLGT', 'dkGXS', 'wJwGT', 'YZQPT', 'yFtyhb', 

In [32]:
jobid = 'rExkeb'

check_jobinfo = requests.get(
    apibaseurl + '/api/v2/jobs/' + jobid,
    headers = {'Authorization': token}
)
print('Get the job information is done')

check_jobinfo_dict = json.loads(check_jobinfo.text)
json_jobinfo = json.dumps(check_jobinfo_dict, indent=4)
with open('info_' + jobid + '.json', "w") as file:
    file.write(json_jobinfo)

Get the job information is done
